### Part 1 - get dataframe of postalcodes per neighborhoods

In [2]:
import pandas as pd
#!conda install beautifulsoup4
from bs4 import BeautifulSoup
import requests


url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # wiki page containing a table of Toronto neighborhoods
r = requests.get(url) 
soup = BeautifulSoup(r.content,"html5lib")  # html content of the url
tables = soup.find('table', class_ = 'wikitable sortable') # get table
rows = tables.find_all('tr') # get table rows

postcodes = []
boroughs = []
neighborhoods = []

for row in rows:
    col = row.find_all('td')
    try:
        if col[1].text != 'Not assigned': # skip rows with not assigned
            postcodes.append(col[0].text)
            boroughs.append(col[1].text)
            nbds_tmp = col[2].text.split('\n')[0] # remove new line tag
            if nbds_tmp == 'Not assigned': # if neighborhood isnt assigned, use borough instead
                neighborhoods.append(col[1].text)
            else:
                neighborhoods.append(nbds_tmp)    
    except: # first row
        pass
    

d = {'Postcode': postcodes, 'Borough': boroughs ,'Neighborhood': neighborhoods}
tor_nbds = pd.DataFrame(data=d)
tor_nbds = tor_nbds.groupby(['Postcode','Borough'])['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index()
tor_nbds.head()
print('Number of rows: ', tor_nbds.shape[0])
print('Number of cols: ', tor_nbds.shape[1])

Number of rows:  103
Number of cols:  3


### Part 2 - get coordinates per postal code

In [3]:
#!pip install pgeocode
import pgeocode

pcodes = tor_nbds['Postcode'].values # get a list of postcodes
nomi = pgeocode.Nominatim('ca') # Get info on all neighborhoods in Canada
lat = []
lon = []

for pcode in pcodes: # loop over each postcode and get the coordinates
    pcode_info = nomi.query_postal_code(pcode)
    lat.append(pcode_info.latitude)
    lon.append(pcode_info.longitude)

tor_nbds['Latitude'] = lat
tor_nbds['Longitude'] = lon

# postcode in line 86 missing in pgeocode data
tor_nbds.loc[86,'Latitude'] = 43.6369
tor_nbds.loc[86,'Longitude'] = -79.615819

tor_nbds.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8113,-79.1930
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


### Part 3 - cluster neighborhoods

Create a map of Toronto with the neighborhoods on top

In [6]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

latitude_tr = 43.65
longitude_tr = -79.38

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude_tr, longitude_tr], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor_nbds['Latitude'], tor_nbds['Longitude'], tor_nbds['Borough'], tor_nbds['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare credential

In [23]:
# @hidden_cell

CLIENT_ID = 'UMGC5P4NT0BFLKOWQW4VYYE0JNO1YKRAWHR1VJKJM5BKC1CX' # your Foursquare ID
CLIENT_SECRET = 'T2V0ISAIFQF0RXNG2MJTBBLK1WOEWYLDWOQTXFGB4EFB0B54' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Create a function that returns venues per neighborhood (in a radius of 500 m)

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Use the function above to create a new dataframe that contains all Toronto neighborhoods, venues, category and coordinates

In [9]:
tor_venues = getNearbyVenues(names=tor_nbds['Neighborhood'],
                                   latitudes=tor_nbds['Latitude'],
                                   longitudes=tor_nbds['Longitude']
                                  )
tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.8113,-79.1930,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Highland Creek, Rouge Hill, Port Union",43.7878,-79.1564,Fox and Fiddle,43.789082,-79.154459,Bar
2,"Guildwood, Morningside, West Hill",43.7678,-79.1866,Chick-N-Joy,43.768752,-79.187982,Fried Chicken Joint
3,"Guildwood, Morningside, West Hill",43.7678,-79.1866,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
4,"Guildwood, Morningside, West Hill",43.7678,-79.1866,LCBO,43.771462,-79.184384,Liquor Store


Display how many venues per neighborhood

In [10]:
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",1,1,1,1,1,1
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Downsview North, Wilson Heights",7,7,7,7,7,7
Bayview Village,3,3,3,3,3,3
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,92,92,92,92,92,92


How many unique categories are?

In [11]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 268 uniques categories.


Use one hot encoding to analyze each neighborhood by venue category

In [12]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot.insert(0, 'Neighborhood', tor_venues['Neighborhood'], True )
tor_onehot = tor_onehot.loc[:, ~tor_onehot.columns.duplicated()] # remove duplicate columns

# group by neighborhood
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()

tor_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.03,0.01,0.0,0.03,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


Create a dataframe with the top 10 venues per neighborhood

In [13]:
# This function returns the top number of categories 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Restaurant,Steakhouse,Gym,Gastropub,Hotel,Bar,Salad Place,Asian Restaurant
1,Agincourt,Latin American Restaurant,Breakfast Spot,Badminton Court,Skating Rink,Shanghai Restaurant,Yoga Studio,Field,Falafel Restaurant,Farmers Market,Fast Food Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Yoga Studio,Field,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Auto Garage,Fried Chicken Joint,Beer Store,Hardware Store,Fast Food Restaurant,Sandwich Place,Pizza Place,Pharmacy,Financial or Legal Service
4,"Alderwood, Long Branch",Pizza Place,Pub,Coffee Shop,Gym,Sandwich Place,Dance Studio,Pharmacy,Convenience Store,Field,Fast Food Restaurant


#### Clustering

Running k-means to cluster neighborhoods, and creating a df combining tor_nbds df and neighborhoods_venues_sorted df

In [15]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


tor_merged = tor_nbds

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged = tor_merged.dropna()
tor_merged.reset_index(drop=True)

tor_merged['Cluster Labels'] = tor_merged['Cluster Labels'].astype('int64') 
tor_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.8113,-79.1930,2,Home Service,Yoga Studio,Field,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7878,-79.1564,3,Bar,Field,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fountain
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866,0,Pizza Place,Grocery Store,Fast Food Restaurant,Coffee Shop,Pharmacy,Electronics Store,Supermarket,Fried Chicken Joint,Beer Store,Mexican Restaurant
3,M1G,Scarborough,Woburn,43.7712,-79.2144,2,Convenience Store,Korean Restaurant,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fountain
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389,2,Construction & Landscaping,Trail,Lounge,Gaming Cafe,Yoga Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market


Count how many neighborhoods in each cluster

In [16]:
tor_merged['Cluster Labels'].value_counts()

0    67
4    18
2    12
1     2
3     1
Name: Cluster Labels, dtype: int64

Visualize the clusters

In [17]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude_tr, longitude_tr], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters

Cluster 1

In [18]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0,Pizza Place,Grocery Store,Fast Food Restaurant,Coffee Shop,Pharmacy,Electronics Store,Supermarket,Fried Chicken Joint,Beer Store,Mexican Restaurant
5,Scarborough,0,Grocery Store,Dumpling Restaurant,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Financial or Legal Service,Field,Fast Food Restaurant
6,Scarborough,0,Coffee Shop,Discount Store,Pharmacy,Department Store,Metro Station,Light Rail Station,Sandwich Place,Bank,Intersection,Bus Station
7,Scarborough,0,Bus Line,Intersection,Bakery,Soccer Field,Bus Station,Park,Metro Station,Coffee Shop,Yoga Studio,Fast Food Restaurant
8,Scarborough,0,Fast Food Restaurant,Pizza Place,Liquor Store,Bank,Coffee Shop,Discount Store,Bistro,Pharmacy,Flower Shop,Sandwich Place
9,Scarborough,0,College Stadium,General Entertainment,Skating Rink,Café,Field,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Yoga Studio
12,Scarborough,0,Latin American Restaurant,Breakfast Spot,Badminton Court,Skating Rink,Shanghai Restaurant,Yoga Studio,Field,Falafel Restaurant,Farmers Market,Fast Food Restaurant
13,Scarborough,0,Pizza Place,Fried Chicken Joint,Bank,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Bus Stop,Pharmacy,Thai Restaurant,Fish & Chips Shop
15,Scarborough,0,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Coffee Shop,Grocery Store,Gym,Other Great Outdoors,Sandwich Place,Bubble Tea Shop,Pharmacy
18,North York,0,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Coffee Shop,Toy / Game Store,Baseball Field,Tea Room,Food Court,Women's Store,Bookstore


Cluster 2

In [19]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,1,Bakery,Asian Restaurant,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service
79,North York,1,Bakery,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Eastern European Restaurant


Cluster 3

In [20]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Home Service,Yoga Studio,Field,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service
3,Scarborough,2,Convenience Store,Korean Restaurant,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fountain
4,Scarborough,2,Construction & Landscaping,Trail,Lounge,Gaming Cafe,Yoga Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
11,Scarborough,2,Convenience Store,Intersection,Auto Garage,Yoga Studio,Financial or Legal Service,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
14,Scarborough,2,Pharmacy,Yoga Studio,Field,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service
20,North York,2,Cafeteria,Pool,Yoga Studio,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service
21,North York,2,Playground,Bus Line,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Financial or Legal Service,Field
31,North York,2,Pool,Moving Target,Yoga Studio,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Field
32,North York,2,Baseball Field,Field,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Yoga Studio,Fountain
36,East York,2,Convenience Store,Asian Restaurant,Beer Store,Video Store,Spa,Flower Shop,Fish Market,Food,Food & Drink Shop,Electronics Store


Cluster 4

In [21]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Bar,Field,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fountain


Cluster 5

In [22]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,4,Park,Residential Building (Apartment / Condo),Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Yoga Studio,Dumpling Restaurant
19,North York,4,Park,Construction & Landscaping,Trail,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Yoga Studio
23,North York,4,Park,Convenience Store,Bank,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Yoga Studio
24,North York,4,Coffee Shop,Gym / Fitness Center,Baby Store,Park,Yoga Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
25,North York,4,Construction & Landscaping,Food & Drink Shop,Park,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
26,North York,4,Park,Pool,Basketball Court,Flower Shop,Food,Fish Market,Fish & Chips Shop,Financial or Legal Service,Field,Dumpling Restaurant
27,North York,4,Gym,River,Park,Yoga Studio,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant
40,East York,4,Park,Convenience Store,Coffee Shop,Rental Car Location,Massage Studio,Hobby Shop,Doner Restaurant,Fish Market,Fish & Chips Shop,Financial or Legal Service
44,Central Toronto,4,Photography Studio,Lawyer,Park,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant
45,Central Toronto,4,Clothing Store,Food & Drink Shop,Breakfast Spot,Gym,Park,Dog Run,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant
